# Capstone Project - Starbucks Site selection in Toronto
### IBM Data Science Professional Certificate by Coursera

## Contents

1. [Introduction: Business Problem](#introduction)
2. [Data](#data)

## 1. Introduction: Business Problem <a name="introduction"></a>

Given the recent announcement in early 2021 of Starbucks closing down up to 300 coffee shops across Canada in response to a change in customer behaviour and preferences, the company is closing stores in downtown core areas and will be focusing on expanding to more pick-up and convenience-led store formats. The stakeholders of Starbucks corp. are looking for optimal sites to open their next stores to reduce cost and obtain high benefits.

Toronto is the capital city of the Canadian province of Ontario and the most populous city in Canada. Unlike most of the grid-plan suburbs found in the outskirts of most North American cities, many suburban neighborhoods in Toronto encouraged high-density populations by mixing single-detached housing with higher-density apartment blocks. This kind of diverse cityscape provides ample opportunites to open the new-format Starbucks.

Socioeconomic factors such as population, regional income level, consumer demographics (age) and competitors are important influencers in a good site selection strategy. The objective of this study is to generate a few most promising neighborhoods in Toronto based on these factors. We will use various Data Science techniques including Clustering to explore the neighborhoods, cluster them based on the above factors, compare to the existing Starbucks locations, and select the optimal neighborhood cluster(s) to open the new-format Starbucks. 

## 2. Data <a name="data"></a>

### 2.1. Data Sources <a name="data sources"></a>

Based on our business problem, the factors that will influence our decision are:
   * population
   * income
   * age groups
   * number of competitors (coffee shops)

Following data sources will be needed to extract/generate the required information:

a) https://open.toronto.ca/dataset/neighbourhood-profiles/: The population demographics information (population and income of each neighborhoods) will be obtained from this link of **Open Data Portal - City of Toronto**.

b) https://open.toronto.ca/dataset/wellbeing-toronto-demographics/ : Population grouped by age groups in each neighborhood will be generated from this link of **Open Data Portal - City of Toronto**.

c) https://opencagedata.com : The **OpenCage Geocoding API** will provide the geographical coordinates of the neighborhoods.

d) https://developer.foursquare.com/ : Number of coffee shops, their locations and that of Starbucks will be extracted using the explore function of the **Foursquare API**. 


### 2.2. Data Cleaning <a name="data cleaning"></a>

### Creating Neighborhoods profile dataframe

#### Load population demographics data

In [2]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [4]:
%cd "E:\Studies\Coursera\IBM Data Science Prof. Cert\Lab\Course 10 Applied Data Science Capstone project\Capstone Project"

E:\Studies\Coursera\IBM Data Science Prof. Cert\Lab\Course 10 Applied Data Science Capstone project\Capstone Project


In [5]:
#load the neighborhood profile dataset and create a dataframe
df_pop = pd.read_csv('neighbourhood_profiles.csv')
df_pop.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [6]:
#drop columns that is not needed
df_pop.drop(df_pop.columns[[0,1,2,3]], axis=1, inplace=True)
df_pop.set_index('Characteristic', inplace=True)

#slice to get a dataframe. we are interested in population and after-tax income
df_pop_sliced = df_pop.loc[["Population, 2016",
    "Total - After-tax income groups in 2015 for the population aged 15 years and over in private households - 100% data"]]
df_pop_sliced.drop(['City of Toronto'], axis=1, inplace=True)

#transpose the dataframe and rename the columns
df_neigh = df_pop_sliced.T
df_neigh.columns = ['Population', 'Income']
df_neigh.index.name = 'Neighborhood'

print(df_neigh.shape)
df_neigh.head()

,Population,Income
Neighborhood,,
Agincourt North,"29,113","24,995"
Agincourt South-Malvern West,"23,757","20,395"
Alderwood,"12,054","10,265"
Annex,"30,526","26,305"
Banbury-Don Mills,"27,695","23,390"


#### Load age group data

In [7]:
#load age group data
df_age = pd.read_excel('age_population.xlsx')

#drop neighbourhoodID and rename column
df_age.drop(['NeighbourhoodID', 'Total Population - All Age Groups - 100% data'], axis=1, inplace=True)
df_age.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_age.set_index(['Neighborhood'])

print(df_age.shape)
df_age.head()

(141, 25)


,Neighborhood,0 to 04 years,0 to 14 years,05 to 09 years,10 to 14 years,100 years and over,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,...,60 to 64 years,65 to 69 years,65 years and over,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,85 years and over,90 to 94 years,95 to 99 years
0,West Humber-Clairville,1540.0,5060.0,1720.0,1790.0,5.0,2325.0,3120.0,2785.0,2345.0,...,1795.0,1595.0,4980.0,1185.0,885.0,700.0,400.0,615.0,160.0,50.0
1,Mount Olive-Silverstone-Jamestown,2190.0,7090.0,2500.0,2415.0,0.0,2585.0,2655.0,2400.0,2250.0,...,1520.0,1285.0,3560.0,885.0,630.0,465.0,225.0,300.0,70.0,10.0
2,Thistletown-Beaumond Heights,540.0,1730.0,600.0,595.0,5.0,650.0,760.0,680.0,715.0,...,535.0,490.0,1880.0,375.0,335.0,320.0,225.0,350.0,100.0,20.0
3,Rexdale-Kipling,560.0,1640.0,515.0,565.0,0.0,635.0,720.0,715.0,680.0,...,650.0,520.0,1730.0,350.0,295.0,270.0,205.0,300.0,85.0,15.0
4,Elms-Old Rexdale,540.0,1805.0,605.0,660.0,0.0,690.0,750.0,600.0,575.0,...,525.0,415.0,1275.0,305.0,235.0,180.0,105.0,145.0,40.0,5.0


#### Merge the two dataframes

In [9]:
df_census = pd.merge(df_neigh, df_age, on = 'Neighborhood', how = 'inner')

# drop some columns
df_census.drop(['0 to 04 years','05 to 09 years', '100 years and over', '55 years and over', '65 years and over', 
                '85 years and over'], axis=1, inplace=True)

#add a new column "State" contaning the value Ontario to dataframe 
df_census["State"] = 'Ontario'

# change population and income to float

df_census['Population'] = df_census['Population'].str.replace(',', '').astype(float)
df_census['Income'] = df_census['Income'].str.replace(',', '').astype(float)

print(df_census.shape)
df_census.head()

(139, 22)


,Neighborhood,Population,Income,0 to 14 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,...,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,90 to 94 years,95 to 99 years,State
0,Agincourt North,29113.0,24995.0,3840.0,1240.0,1705.0,2000.0,2020.0,1775.0,1465.0,...,2230.0,2000.0,1905.0,1290.0,1065.0,865.0,565.0,270.0,90.0,Ontario
1,Agincourt South-Malvern West,23757.0,20395.0,3075.0,935.0,1470.0,1890.0,2020.0,1645.0,1340.0,...,1755.0,1510.0,1320.0,875.0,760.0,595.0,365.0,155.0,35.0,Ontario
2,Alderwood,12054.0,10265.0,1760.0,480.0,570.0,665.0,715.0,840.0,905.0,...,1030.0,795.0,620.0,420.0,340.0,315.0,190.0,100.0,15.0,Ontario
3,Annex,30526.0,26305.0,2360.0,675.0,1015.0,2735.0,4350.0,3295.0,2090.0,...,1780.0,1700.0,1725.0,1335.0,1030.0,750.0,570.0,340.0,110.0,Ontario
4,Banbury-Don Mills,27695.0,23390.0,3605.0,1285.0,1370.0,1360.0,1400.0,1595.0,1625.0,...,1935.0,1620.0,1670.0,1360.0,1235.0,1085.0,955.0,520.0,125.0,Ontario


#### Get geographical coordinates using OpenCage API

In [10]:
#install and import OpenCageGeocode module
!pip install opencage

from opencage.geocoder import OpenCageGeocode

print("Libraries loaded!")

Libraries loaded!


As the simplest, not-most-efficient approach, I am going to iterate over each row to get the city and state, then use the API to get the corresponding coordinates. I’ll save longitudes and latitudes in two separate lists. Then I can add these two lists as new columns once I’m done.

In [11]:
key = 'b87f2c9f6e4c426caf738d3b6c23a9c4' #API key from https://opencagedata.com

geocoder = OpenCageGeocode(key)

list_lat = [] # create empty lists
list_long = []

for index, row in df_census.iterrows(): #iterate over rows in dataframe
    
    Neighborhood = row['Neighborhood']
    State = row['State']
    query = str(Neighborhood)+','+str(State)
    
    results = geocoder.geocode(query)
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    
    list_lat.append(lat)
    list_long.append(long)

# Create new columns from lists
df_census['Latitude'] = list_lat
df_census['Longitude'] = list_long

In [12]:
print(df_census.shape)
df_census.head() #check the last columns!

(139, 24)


,Neighborhood,Population,Income,0 to 14 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,...,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,90 to 94 years,95 to 99 years,State,Latitude,Longitude
0,Agincourt North,29113.0,24995.0,3840.0,1240.0,1705.0,2000.0,2020.0,1775.0,1465.0,...,1905.0,1290.0,1065.0,865.0,565.0,270.0,90.0,Ontario,43.808038,-79.266439
1,Agincourt South-Malvern West,23757.0,20395.0,3075.0,935.0,1470.0,1890.0,2020.0,1645.0,1340.0,...,1320.0,875.0,760.0,595.0,365.0,155.0,35.0,Ontario,43.788555,-79.265661
2,Alderwood,12054.0,10265.0,1760.0,480.0,570.0,665.0,715.0,840.0,905.0,...,620.0,420.0,340.0,315.0,190.0,100.0,15.0,Ontario,43.601717,-79.545232
3,Annex,30526.0,26305.0,2360.0,675.0,1015.0,2735.0,4350.0,3295.0,2090.0,...,1725.0,1335.0,1030.0,750.0,570.0,340.0,110.0,Ontario,43.670338,-79.407117
4,Banbury-Don Mills,27695.0,23390.0,3605.0,1285.0,1370.0,1360.0,1400.0,1595.0,1625.0,...,1670.0,1360.0,1235.0,1085.0,955.0,520.0,125.0,Ontario,43.752339,-79.365716


Now we have our Neighborhoods Profile Dataframe. The dataframe contains the list of neighborhoods in Toronto along with their population, income, population per age group, and geo coordinates. 

Let's visualize the data.

In [22]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [23]:
#Use OpenCageGeocode library to get the latitude and longitude values of Toronto

query_toronto = 'Toronto, Ontario'

location_toronto = geocoder.geocode(query_toronto)

latitude_toronto = location_toronto[0]['geometry']['lat']
longitude_toronto = location_toronto[0]['geometry']['lng']

print('The geographical coordinates of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


In [24]:
#Create a map of Toronto with neighborhoods superimposed on top
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

#add markers to map
for lat, lng, neighborhood in zip(df_census['Latitude'], df_census['Longitude'], 
                                         df_census['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Foursquare API - extracting venues

Let's explore each neighborhood using Foursquare API to get info on Starbucks and coffee shops. Coffee shops are the direct competitors so they are included.

In [19]:
#Define Foursquare credentials and Version
CLIENT_ID = 'CAFGEUTLP40HWRS2HYMCJC5XSAEHY4N04UB3PQL2WXP4UC2I' #your Foursquare ID
CLIENT_SECRET = 'HYFQMVIQZMJ0HH35PGQY3VMYRO3DQ2LKZAWCWPV42HJHI1RM' #your Foursquare Secret
VERSION = '20190425' #Foursquare API version
LIMIT = 100 #A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET' + CLIENT_SECRET)

Your credentials:
CLIENT_ID:CAFGEUTLP40HWRS2HYMCJC5XSAEHY4N04UB3PQL2WXP4UC2I
CLIENT_SECRETHYFQMVIQZMJ0HH35PGQY3VMYRO3DQ2LKZAWCWPV42HJHI1RM


In [20]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Create a function to get nearby venues to the neighborhoods in Toronto
def getNearbyVenues(neighborhood, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for neighborhood, lat, lng in zip(neighborhood, latitudes, longitudes):
            
        #create the API request URL
        url_foursquare = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_foursquare = requests.get(url_foursquare).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_foursquare])

    #convert the list into a new dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [21]:
#Now apply the above function on each neighborhood and create a new dataframe
df_nearbyvenues = getNearbyVenues(neighborhood=df_census['Neighborhood'],
                                   latitudes=df_census['Latitude'],
                                   longitudes=df_census['Longitude']
                                  )

print(df_nearbyvenues.shape)
df_nearbyvenues.head()

(2628, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.808038,-79.266439,Menchie's,43.808338,-79.268288,Frozen Yogurt Shop
1,Agincourt North,43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
2,Agincourt North,43.808038,-79.266439,Booster Juice,43.809915,-79.269382,Juice Bar
3,Agincourt North,43.808038,-79.266439,Shoppers Drug Mart,43.808894,-79.269854,Pharmacy
4,Agincourt North,43.808038,-79.266439,Dollarama,43.808894,-79.269854,Discount Store


#### Starbucks Data

Filter the above dataframe to get neighborhoods with atleast one Starbucks.

In [25]:
df_starbucks = df_nearbyvenues.query('Venue == "Starbucks"')

print(df_starbucks.shape)
df_starbucks.head()

(41, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
193,Bay Street Corridor,43.664213,-79.387383,Starbucks,43.668071,-79.388367,Coffee Shop
219,Bedford Park-Nortown,43.731516,-79.420191,Starbucks,43.732604,-79.419136,Coffee Shop
221,Bedford Park-Nortown,43.731516,-79.420191,Starbucks,43.728673,-79.418513,Coffee Shop
262,Blake-Jones,43.676170,-79.337378,Starbucks,43.679850,-79.340370,Coffee Shop
370,Church-Yonge Corridor,43.670786,-79.385687,Starbucks,43.670340,-79.388262,Coffee Shop


Merge the starbucks dataframe with the census dataframe to get the final dataframe consisting of details of the neighborhoods having Starbucks.

In [26]:
df_starbucks_final = pd.merge(df_census, df_starbucks, on = 'Neighborhood', how = 'inner')

print(df_starbucks_final.shape)
df_starbucks_final.head()

(41, 30)


,Neighborhood,Population,Income,0 to 14 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,...,95 to 99 years,State,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Street Corridor,25797.0,23940.0,1695.0,240.0,1385.0,5475.0,4545.0,3330.0,1840.0,...,10.0,Ontario,43.664213,-79.387383,43.664213,-79.387383,Starbucks,43.668071,-79.388367,Coffee Shop
1,Bedford Park-Nortown,23236.0,18560.0,4555.0,1705.0,1725.0,1485.0,875.0,1095.0,1370.0,...,45.0,Ontario,43.731516,-79.420191,43.731516,-79.420191,Starbucks,43.732604,-79.419136,Coffee Shop
2,Bedford Park-Nortown,23236.0,18560.0,4555.0,1705.0,1725.0,1485.0,875.0,1095.0,1370.0,...,45.0,Ontario,43.731516,-79.420191,43.731516,-79.420191,Starbucks,43.728673,-79.418513,Coffee Shop
3,Blake-Jones,7727.0,6280.0,1405.0,395.0,450.0,435.0,495.0,615.0,670.0,...,10.0,Ontario,43.676170,-79.337378,43.676170,-79.337378,Starbucks,43.679850,-79.340370,Coffee Shop
4,Church-Yonge Corridor,31340.0,29095.0,1260.0,270.0,1040.0,4020.0,5540.0,4485.0,2750.0,...,5.0,Ontario,43.670786,-79.385687,43.670786,-79.385687,Starbucks,43.670340,-79.388262,Coffee Shop


Visualize starbucks locations on a map.

In [27]:
# create map
map_starbucks = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to the map
for lat, lon, poi, pop, inc in zip(df_starbucks_final['Venue Latitude'], df_starbucks_final['Venue Longitude'], df_starbucks_final['Neighborhood'], df_starbucks_final['Population'], df_starbucks_final['Income']):
    label = folium.Popup('{}, Pop: {}, Inc: {}'.format(poi, pop, inc), parse_html=True)    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='starbucks green',
        fill=True,
        fill_color='#00704A',
        fill_opacity=0.7).add_to(map_starbucks)
       
map_starbucks

As we mentioned previously in Introduction: Business Problem section, Starbucks is mainly concentrated in the core Downtown of Toronto. The visualization concurs with our problem and is good for our analysis to move outwards for new location.